In [1]:
import pandas as pd
import nltk
import spacy
import gensim
import seaborn as sb
from gensim import corpora, models, similarities
from spacy.lang.en import English
from nltk.corpus import wordnet as wn

In [2]:
data=pd.read_csv("MecklenburgCountyOriginalDataFY20.csv",)

In [3]:
data.head()

,Unnamed: 0,page_number,word
0,1,1,meckncgov
1,2,1,fiscal
2,3,1,year
3,4,1,adopted
4,5,1,budget


In [4]:
data.drop(data.columns[0], axis=1)

,page_number,word
0,1,meckncgov
1,1,fiscal
2,1,year
3,1,adopted
4,1,budget
5,1,mecklenburg
6,1,county
7,1,north
8,1,carolina
9,2,adopted


In [5]:
data.describe()

,Unnamed: 0,page_number
count,84235.0000,84235.000000
mean,42118.0000,260.653505
std,24316.6943,171.952653
min,1.0000,1.000000
25%,21059.5000,98.000000
50%,42118.0000,252.000000
75%,63176.5000,416.000000
max,84235.0000,537.000000


In [6]:
#cleaning 
spacy.load('en')
parser=English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.orth_.startswith('http'):
            lda_tokens.append('com')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
def get_lemma(word):
    lemma=wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [9]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [10]:
#filtering stopwords
nltk.download('stopwords')
stop=set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#preparing for lda
def prepare_lda(text):
    tokens=tokenize(text)
    tokens=[token for token in tokens if len(token)>4]
    tokens=[token for token in tokens if token not in stop]
    tokens=[get_lemma(token) for token in tokens]
    return tokens

In [12]:
#creating tokens
import random
text_data=[]
with open('MecklenburgCountyOriginalDataFY20.csv') as f:
    for line in f:
        tokens=prepare_lda(line)
        if random.random()>.99:
            print(tokens)
            text_data.append(tokens)

['10",2,"adopted']
['16",2,"distinguished']
['104",2,"and']
['200",3,"janice']
['450",6,"county']
['672",6,"community']
['766",6,"revenues']
['792",6,"and']
['883",7,"nondepartmental']
['887",7,"for']
['957",7,"fy']
['1256",9,"commitment']
['1411",9,"budget']
['1438",9,"an']
['1468",10,"revenues']
['1520",10,"need']
['1626",10,"that']
['1776",10,"staffing']
['1896",10,"addition']
['1909",10,"inspections']
['1921",11,"funding']
['2015",11,"the']
['2048",11,"support']
['2099",11,"and']
['2201",11,"increase']
['2288",11,"payments']
['2326",11,"will']
['2333",11,"to']
['2334",11,"care']
['2336",11,"these']
['2394",11,"year']
['2476",12,"assigned']
['2543",12,"just']
['2598",12,"supplement']
['2746",12,"year']
['2784",12,"some']
['2856",12,"county']
['2903",13,"home']
['3029",13,"charlotte']
['3072",13,"to']
['3083",13,"others']
['3144",13,"in']
['3178",13,"be']
['3217",13,"such']
['3277",13,"a']
['3312",14,"budget']
['3422",14,"community']
['3567",14,"the']
['3597",14,"program']
['3735",14

['36005",198,"revenue']
['36061",199,"fy']
['36229",201,"goals']
['36416",203,"process']
['36453",203,"overview']
['36564",204,"expenses']
['36781",207,"service']
['36829",208,"type']
['36886",208,"pt']
['36997",209,"and']
['37107",210,"approved']
['37269",212,"countys']
['37372",212,"expenses']
['37394",212,"ft']
['37462",213,"adopted']
['37635",214,"adopted']
['37829",216,"designation']
['37908",217,"goals']
['37920",217,"funding']
['37934",217,"to']
['38021",219,"type']
['38040",219,"adopted']
['38101",220,"to']
['38319",221,"efforts']
['38393",221,"in']
['38454",223,"compensation']
['38559",223,"fy']
['38602",223,"ft']
['38653",224,"their']
['38832",225,"number']
['38891",225,"basis']
['38969",226,"services']
['38999",226,"technical']
['39120",227,"lpt']
['39291",229,"data']
['39398",230,"systems']
['39430",230,"the']
['39531",231,"description']
['39610",231,"to']
['39616",231,"services']
['39696",232,"management']
['39804",233,"of']
['39837",233,"requisite']
['39916",233,"resoluti

['70248",478,"significant']
['70265",478,"with']
['70358",478,"the']
['70378",478,"but']
['70405",478,"and']
['70417",478,"library']
['70430",478,"learning']
['70550",480,"approved']
['70583",480,"which']
['70591",480,"timeline']
['70676",480,"carolina']
['70890",481,"creek']
['70927",481,"lakeview']
['70946",481,"ride']
['70990",482,"and']
['71058",482,"ordinances']
['71170",489,"adopted']
['71232",495,"recommended']
['71393",496,"programs']
['71415",496,"ncgs']
['71608",497,"county']
['71629",497,"and']
['71790",497,"community']
['71827",497,"costs']
['71856",497,"possible']
['71882",497,"and']
['71916",498,"adopted']
['72252",498,"appropriation']
['72318",499,"governing']
['72328",499,"amend']
['72418",499,"specified']
['72488",499,"for']
['72696",500,"agency']
['72738",500,"year']
['72938",501,"financial']
['73054",501,"for']
['73156",501,"the']
['73169",501,"requires']
['73206",502,"appropriations']
['73298",502,"becomes']
['73497",502,"for']
['73541",502,"when']
['73649",502,"and

In [13]:
#creating dictionary
from gensim import corpora
dictionary=corpora.Dictionary(text_data)
corpus= [dictionary.doc2bow(text) for text in text_data]
import pickle 
pickle.dump(corpus, open('corpus.pkl', 'wb'))

In [14]:
#example
ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=6)
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} word: {}'.format(idx, topic))

Topic: 0 word: 0.006*"82153",531,"capital" + 0.006*"22695",106,"debt" + 0.006*"49449",309,"out" + 0.006*"63930",423,"services" + 0.006*"60688",395,"lpt" + 0.006*"70378",478,"but" + 0.006*"9239",41,"board" + 0.006*"22980",109,"services" + 0.006*"4420",16,"take" + 0.006*"58048",378,"training"
Topic: 1 word: 0.006*"38969",226,"services" + 0.006*"82842",533,"modified" + 0.006*"39291",229,"data" + 0.006*"1626",10,"that" + 0.006*"9619",43,"holding" + 0.006*"67883",461,"developed" + 0.006*"35960",198,"the" + 0.006*"10435",46,"joint" + 0.006*"78212",513,"also" + 0.006*"61997",404,"fy"
Topic: 2 word: 0.006*"65682",440,"pt" + 0.006*"16853",79,"year" + 0.006*"766",6,"revenues" + 0.006*"77767",512,"form" + 0.006*"10685",48,"county" + 0.006*"70196",478,"of" + 0.006*"7078",27,"to" + 0.006*"2543",12,"just" + 0.006*"48844",305,"pt" + 0.006*"47405",294,"producers"
Topic: 3 word: 0.007*"12443",57,"county" + 0.007*"67157",455,"fy" + 0.007*"79035",516,"in" + 0.007*"23349",110,"debt" + 0.007*"71232",495,"r

In [16]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)